In [1]:
import pandas as pd
import numpy as np
# read the data

from os import walk
f =[]
statepath = "newState_0523/"

# get all filenames in the directory
for (a, b, filenames) in walk(statepath):
     f.extend(filenames) 
     break
print (len(filenames))

# split the filenames
data_contain = {'name': [], 'd_all': [], 'd_0_14':[],'d_15_24': [], 'd_25_34': [], 'd_35_44': [], 
                      'd_45_54': [],  'd_55_64': [], 
                      'd_65_74': [], 'd_75+': [], 'gender':[],'state':[]}

all_data = pd.DataFrame(data_contain) 
# read the datasheet
gender =['female','male','both']

# add the population to the file, population= dict type 
population = {}; 

for g in gender:
    population[g] = pd.read_excel("population.xlsx",sheet_name=g)
    population[g].set_index("state", drop=True, append=False,inplace=True)

for  filename in filenames:
    if filename.split(".")[0] == '':
        continue
    path = statepath+ filename

    # read the state data
    for g in gender:
        state_name = filename.split(".")[0] # generate state name by using filename {statename.xlsx}
        state_data = pd.read_excel (path,sheet_name=g) # read state data
        disease_number = state_data.shape[0] # the row number for this state data = the number of the disease
        state_data['gender'] = [g]* disease_number 
        state_data['state'] = [state_name]* disease_number
        
        state_population= pd.DataFrame(np.repeat((population[g].loc[[state_name],:]).values,disease_number,axis=0)) # read the population data
        state_population.columns = population[g].columns
        state_data = state_data.join(state_population) # join state_data with population data
        
        all_data = all_data.append (state_data,ignore_index = True, sort=False)
        
all_data.head()    

52


,name,d_all,d_0_14,d_15_24,d_25_34,d_35_44,d_45_54,d_55_64,d_65_74,d_75+,...,state,p_all,p_0_14,p_15_24,p_25_34,p_35_44,p_45_54,p_55_64,p_65_74,p_75+
0,Alzheimers,978.0,0.0,0.0,0.0,0.0,0.0,11.0,68.0,899.0,...,Arkansas,1528215.0,285612.0,196314.0,196976.0,184157.0,190072.0,198167.0,154810.0,122107.0
1,Anaemia,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,Arkansas,1528215.0,285612.0,196314.0,196976.0,184157.0,190072.0,198167.0,154810.0,122107.0
2,Appendicitis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Arkansas,1528215.0,285612.0,196314.0,196976.0,184157.0,190072.0,198167.0,154810.0,122107.0
3,Asthma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Arkansas,1528215.0,285612.0,196314.0,196976.0,184157.0,190072.0,198167.0,154810.0,122107.0
4,Bladder Cancer,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,...,Arkansas,1528215.0,285612.0,196314.0,196976.0,184157.0,190072.0,198167.0,154810.0,122107.0


In [2]:
# calculate death rate
age_range="NULL"
ranges = np.array([0]+[(5+10*x) for x in range(1,8)]) # [ 0, 15, 25, 35, 45, 55, 65, 75]
ranges = np.hstack((ranges[:,None],(ranges + 10*np.ones(len(ranges)))[:,None]))
ranges[0,1] = ranges[1,0]
ranges[-1,-1] = -1
all_data['drate_all'] = all_data['d_all']/all_data['p_all']
for rg in ranges:
    s, e = [int(i) for i in rg.tolist()]
    if e> 0:
        age_range = f"{s}_{e-1}"
    else:
        age_range = f"{s}+"
    all_data['drate_'+age_range] = all_data['d_'+age_range]/all_data['p_'+age_range]
all_data.head()

,name,d_all,d_0_14,d_15_24,d_25_34,d_35_44,d_45_54,d_55_64,d_65_74,d_75+,...,p_75+,drate_all,drate_0_14,drate_15_24,drate_25_34,drate_35_44,drate_45_54,drate_55_64,drate_65_74,drate_75+
0,Alzheimers,978.0,0.0,0.0,0.0,0.0,0.0,11.0,68.0,899.0,...,122107.0,0.000640,0.0,0.0,0.0,0.0,0.0,0.000056,0.000439,0.007362
1,Anaemia,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,122107.0,0.000007,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000082
2,Appendicitis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,122107.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,Asthma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,122107.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,Bladder Cancer,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,...,122107.0,0.000022,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000270


In [3]:
all_data.to_csv("state_population.csv", index=False)